In [59]:
import sys, os, imp
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import lambda2nu, nu2lambda, c

In [60]:
# lumapiFile = "/opt/lumerical/v221/api/python/lumapi.py"
lumapiFile = "C:\\Program Files\\Lumerical\\v221\\api\\python\\lumapi.py"
lumapi = imp.load_source("lumapi", lumapiFile)
fdtdApi = lumapi.FDTD(hide = False)

Exception ignored in: <function Lumerical.__del__ at 0x00000194AC04F600>
Traceback (most recent call last):
  File "C:\Program Files\Lumerical\v221\api\python\lumapi.py", line 1196, in __del__
    close(self.handle)
  File "C:\Program Files\Lumerical\v221\api\python\lumapi.py", line 245, in close
    iapi.appClose(handle)
ctypes.ArgumentError: argument 1: TypeError: expected Session instance instead of Session


In [61]:
# units
um = 1e-6
nm = 1e-9

In [62]:
sub_material = 'SiO2 (Glass) - Palik'
core_material = 'Si (Silicon) - Palik'

gap = 200*nm

x_span_core =  10.35*um # para 50/50 otimizado
y_span_core =  450*nm
z_span_core =  220*nm

x_core = 0*um
y_core = y_span_core+gap
z_core = 0.0*um

wg90Raius = 10*um

y_span_guia_reto = 2*um

## Create Acoplador Direcional


In [63]:
fdtdApi.switchtolayout()
fdtdApi.deleteall()

# Adicionando guias retos - acoplador
fdtdApi.addrect()
fdtdApi.set("name", "Lc")
fdtdApi.set("material", core_material)
fdtdApi.set("x", x_core)
fdtdApi.set("x span", x_span_core)
fdtdApi.set("y", y_core)
fdtdApi.set("y span", y_span_core)
fdtdApi.set("z", z_core)
fdtdApi.set("z span", z_span_core)

fdtdApi.copy()
fdtdApi.set("name", "Lc_lower")
fdtdApi.set("y", 0)
fdtdApi.set('x span', x_span_core + wg90Raius*2 + 4*um)

# Adicionando guia bend
fdtdApi.addobject("90_bend_wg")
fdtdApi.set("name", "right")
fdtdApi.set("base height", z_span_core)
fdtdApi.set("base width", y_span_core)
fdtdApi.set("first axis", 'z')
fdtdApi.set('radius', wg90Raius)
fdtdApi.set('rotation 1', -90)
fdtdApi.set('x', x_span_core/2)
fdtdApi.set('y', y_core+wg90Raius)
fdtdApi.set('z', z_core)
fdtdApi.set('material', core_material)

fdtdApi.copy()
fdtdApi.set("name", "left")
fdtdApi.set('rotation 1', 180)
fdtdApi.set('x', -x_span_core/2)

# Adicionando guias retos auxiliares
fdtdApi.addrect()
fdtdApi.set("name", "wg_1")
fdtdApi.set("material", core_material)
fdtdApi.set("x", -(x_span_core + wg90Raius*2)/2)
fdtdApi.set("x span", y_span_core)
fdtdApi.set("y", wg90Raius+y_core+y_span_guia_reto/2)
fdtdApi.set("y span", y_span_guia_reto)
fdtdApi.set("z", z_core)
fdtdApi.set("z span", z_span_core)

fdtdApi.copy()
fdtdApi.set("name", "wg_2")
fdtdApi.set("x", (x_span_core + wg90Raius*2)/2)


## Criação Solver FDTD e monitores


In [64]:
## Banda de operação
wvlngth_start = 1500 * nm
wvlngth_stop = 1600 * nm
nmbFrqPnts = 21

## Parâmetros do solver FDTD
# Tempo de simulação
simulation_time = (np.pi * wg90Raius + x_span_core)*7/ c + 200e-15

# Dimensões do volume do solver
x_FDTD = 0
y_FDTD = 0
x_span_FDTD = x_span_core + wg90Raius*2 + 2*um
y_span_FDTD = wg90Raius + 2*um + 1.5*um
z_span_FDTD = 4*um

# Material
backGroundMaterial = sub_material

# Precisão do mesh
mesh_accuracy = 2

# Condições de fronteira
z_min_bc = "Symmetric" # TE mode

In [65]:
fdtdApi.switchtolayout()
fdtdApi.select('FDTD')
fdtdApi.delete()
fdtdApi.select('monitor')
fdtdApi.delete()


# Adiciona o solver
fdtdApi.addfdtd()
fdtdApi.set("x", x_FDTD)
fdtdApi.set("x span", x_span_FDTD)
fdtdApi.set("y", wg90Raius/2)
fdtdApi.set("y span", y_span_FDTD)
fdtdApi.set("z", 0)
fdtdApi.set("z span", z_span_FDTD)
fdtdApi.set("background material", backGroundMaterial)
fdtdApi.set("mesh accuracy", mesh_accuracy)
fdtdApi.set("simulation time", simulation_time)
fdtdApi.set("z min bc", z_min_bc)

fdtdApi.addprofile();
fdtdApi.set('name', 'monitor');
fdtdApi.set("x", x_FDTD)
fdtdApi.set("x span", x_span_FDTD)
fdtdApi.set("y", wg90Raius/2)
fdtdApi.set("y span", y_span_FDTD)
fdtdApi.set("z", 0)
fdtdApi.set("override global monitor settings", True)
fdtdApi.set("frequency points", nmbFrqPnts)

# fdtdApi.addmovie();
# fdtdApi.set('name', 'movie');
# fdtdApi.set("x", x_FDTD)
# fdtdApi.set("x span", x_span_FDTD)
# fdtdApi.set("y", wg90Raius/1.7)
# fdtdApi.set("y span", y_span_FDTD)
# fdtdApi.set("z", 0)

## Adiciona portas

In [66]:
# Parametros das portas

y_span_port = y_span_core*4
z_span_port = z_span_core*4
# mode_selection = "fundamental TE mode"
injection_axis_1 = "y"
injection_axis_2 = "x"

In [67]:
# Adiciona portas na simulação
fdtdApi.switchtolayout()
fdtdApi.select('FDTD::ports')
fdtdApi.delete()


fdtdApi.addport()
fdtdApi.set("name", "port upper 1")
fdtdApi.set("injection axis", injection_axis_1)
fdtdApi.set("x", -(x_span_core + wg90Raius*2)/2)
fdtdApi.set("y", wg90Raius + 1.2*um)
fdtdApi.set("z", 0)
fdtdApi.set("x span", y_span_port)
fdtdApi.set("z span", z_span_port)
# fdtdApi.set("mode selection", mode_selection)
fdtdApi.set('direction', 'Backward')

fdtdApi.copy()
fdtdApi.set("name", "port upper 2")
fdtdApi.set("x", (x_span_core + wg90Raius*2)/2)

fdtdApi.addport()
fdtdApi.set("name", "port lower 2")
fdtdApi.set("injection axis", injection_axis_2)
fdtdApi.set("x", (x_span_core + wg90Raius*2)/2)
fdtdApi.set("y", 0)
fdtdApi.set("z", 0)
fdtdApi.set("y span", y_span_port)
fdtdApi.set("z span", z_span_port)
# fdtdApi.set("mode selection", mode_selection)
fdtdApi.set('direction', 'Backward')

fdtdApi.copy()
fdtdApi.set("name", "port lower 1")
fdtdApi.set("x", -(x_span_core + wg90Raius*2)/2)
fdtdApi.set('direction', 'Forward')

In [68]:
# Configura a banda de simulação
fdtdApi.setnamed("FDTD", "global source wavelength start", wvlngth_start)
fdtdApi.setnamed("FDTD", "global source wavelength stop", wvlngth_stop)
fdtdApi.setnamed("FDTD::ports", "monitor frequency points", nmbFrqPnts)

In [69]:
fdtdApi.save('real_coupler')
fdtdApi.run()